In [8]:
import pandas as pd                                   # для работы с датафреймом
import numpy as np                                    # для вычислений
import seaborn as sns                                 # для графиков
import matplotlib.pyplot as plt                       # для графиков
from scipy.stats import norm                          # проверка распределения
from scipy import stats                               # статистический анализ
import scipy.stats as stats                           # статистический анализ
from mlxtend.evaluate import permutation_test         # перестановочный тест
import statsmodels.stats.multicomp as mc              # HDS тест для определения групп
import re                                             # для регулярных выражений
from nltk.tokenize import RegexpTokenizer             # библиотека для работы с естественным языком
import nltk                                           # для обработки естественного языка
from nltk import word_tokenize                        # для деления предложений на слова
from nltk.corpus import stopwords                     # для исключения стоп-слов
from wordcloud import WordCloud, STOPWORDS            # для облака слов
from textblob import TextBlob                         # для анализа эмоций


In [2]:
! pip install mlxtend

     |████████████████████████████████| 1.3 MB 864 kB/s 


In [5]:
! pip install wordcloud

     |████████████████████████████████| 220 kB 840 kB/s 
  Created wheel for wordcloud: filename=wordcloud-1.8.1-cp38-cp38-macosx_10_9_x86_64.whl size=158368 sha256=a85f247e3c9bf4c217ee72f54b2685ad83a4dd7508383bb9585c0dd0e40a8a81
  Stored in directory: /Users/anastasia/Library/Caches/pip/wheels/4d/3f/0d/a2ba9b7895c9f1be89018b3141c3df3d4f9c786c882ccfbc3b
Successfully built wordcloud


In [7]:
! pip install textblob

     |████████████████████████████████| 636 kB 887 kB/s 


In [9]:
! pip install pingouin

     |████████████████████████████████| 204 kB 844 kB/s 
     |████████████████████████████████| 801 kB 957 kB/s 
  Created wheel for pingouin: filename=pingouin-0.3.11-py3-none-any.whl size=203323 sha256=7abdaba72b9a01832e3b86233411c5566f22bfa6308e3f5ce32fc60cb76f034e
  Stored in directory: /Users/anastasia/Library/Caches/pip/wheels/8b/ee/b8/8a5db690ab058c8a7cb998153e2091fe1be41ee59f38fc63e3
  Created wheel for littleutils: filename=littleutils-0.2.2-py3-none-any.whl size=7048 sha256=1fb7bf38b63b2cd73822b04f480feacf6875634f9ed321d2d779198757ddcba9
  Stored in directory: /Users/anastasia/Library/Caches/pip/wheels/6a/33/c4/0ef84d7f5568c2823e3d63a6e08988852fb9e4bc822034870a
Successfully built pingouin littleutils


In [10]:
import pingouin as pg               # базовые функции библиотеки
from pingouin import welch_anova    # тест Велча

# Загрузка данных и общий обзор #

In [11]:
db = pd.read_csv ('data/books_tidy_data.csv', parse_dates = ['publication_date'], dtype = { 'decade' : 'object', 'quarter' : 'object' , 'tra_co' : 'object' })
# загружаем данные и исправляем тип данных

In [12]:
db.head()

,publication_date,title,authors,average_rating,language_code,num_pages,ratings_count,text_reviews_count,publisher,decade,quarter,tra_co
0,2006-09-16,Harry Potter and the Half-Blood Prince (Harry ...,"J.K. Rowling,Mary GrandPré",4.57,eng,652,2095690,27591,Scholastic Inc.,2000,3,1
1,2004-09-01,Harry Potter and the Order of the Phoenix (Har...,"J.K. Rowling,Mary GrandPré",4.49,eng,870,2153167,29221,Scholastic Inc.,2000,3,1
2,2003-11-01,Harry Potter and the Chamber of Secrets (Harry...,J.K. Rowling,4.42,eng,352,6333,244,Scholastic,2000,4,0
3,2004-05-01,Harry Potter and the Prisoner of Azkaban (Harr...,"J.K. Rowling,Mary GrandPré",4.56,eng,435,2339585,36325,Scholastic Inc.,2000,2,1
4,2004-09-13,Harry Potter Boxed Set Books 1-5 (Harry Potte...,"J.K. Rowling,Mary GrandPré",4.78,eng,2690,41428,164,Scholastic,2000,3,1


In [13]:
db = db.set_index ('publication_date') # установка даты в качестве индекса

In [14]:
db.info() # общий обзор данных

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 10838 entries, 2006-09-16 to 2006-05-28
Data columns (total 11 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   title               10838 non-null  object 
 1   authors             10838 non-null  object 
 2   average_rating      10838 non-null  float64
 3   language_code       10838 non-null  object 
 4   num_pages           10838 non-null  int64  
 5   ratings_count       10838 non-null  int64  
 6   text_reviews_count  10838 non-null  int64  
 7   publisher           10838 non-null  object 
 8   decade              10838 non-null  object 
 9   quarter             10838 non-null  object 
 10  tra_co              10838 non-null  object 
dtypes: float64(1), int64(3), object(7)
memory usage: 1016.1+ KB
